# Tubes Mata Kuliah Kecerdasan Buatan

## kelompok 2 : "Prediksi Prestasi Akademik Siswa Menggunakan Machine Learning Berdasarkan Faktor Demografis dan Kehadiran"

Link Dataset: https://www.kaggle.com/datasets/waqi786/student-performance-dataset/data

Anggota :
1. Rakasya Yoga
2. R. Rafi Yudi Pramana
3. Datu reksa Hamza Putra
4. M. Ali Abdillah Khotami
5. Aditya

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

Kode di atas digunakan untuk mengimport library-library yang dibutuhkan, seperti numpy untuk perhitungan numerik (array), pandas untuk dataframe, plt dan sns untuk visualisasi, dan sklearn untuk modelling

In [ ]:
df = pd.read_csv("../dataset/student_performance_data2.csv")
df.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M


Kode di atas digunakan untuk membaca dataset yang akan digunakan, yaitu student_performance_data.csv. Kemudian 5 data teratas ditampilkan menggunakan head.

dalam data tersebut terdapat Kolom-kolom sebagai berikut:

1. sex: Jenis kelamin mahasiswa (0 = perempuan, 1 = laki-laki)  
2. Age: Usia mahasiswa  
3. StudyHoursPerWeek: Jumlah jam belajar per minggu  
4. AttendanceRate: Persentase kehadiran mahasiswa di kelas  
5. PartTimeJob: Status pekerjaan paruh waktu (0 = tidak bekerja, 1 = bekerja)  
6. ExtraCurricularActivities: Keterlibatan dalam kegiatan ekstrakurikuler (0 = tidak ikut, 1 = ikut)
7. GPA: Indeks prestasi kumulatif mahasiswa  



## Data Exploration

In [43]:
# # Konversi GPA menjadi kategori sesuai aturan yang diberikan
# def categorize_gpa(value):
#     if value > 3.8:
#         return "4.0"
#     elif value > 3.2:
#         return "3.5"
#     elif value > 2.8:
#         return "3.0"
#     elif value > 2.2:
#         return "2.5"
#     else:
#         return "2.0"

# # Terapkan fungsi ke kolom GPA
# y = df["GPA"].apply(categorize_gpa)

# # # y.unique()
# # X.head()
# # y.value_counts()

In [ ]:
# FEATURE SELECTION
X = df.drop("GPA", axis=1)  # hapus features
X = X.drop("GradeClass", axis=1)  # hapus features
y = df["GradeClass"].astype(str)  # kelas label

# DATA SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ValueError: could not convert string to float: 'M'

In [45]:
# MODEL TRAINING
# Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)
pred_nb = model_nb.predict(X_test_scaled)

# Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)
pred_dt = model_dt.predict(X_test)

# SVM
model_svm = SVC(kernel='rbf', C=1, gamma='scale')
model_svm.fit(X_train_scaled, y_train)
pred_svm = model_svm.predict(X_test_scaled)

# Random Forest
model_rf = RandomForestClassifier(n_estimators=150, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)


## MODEL PERFORMANCE COMPARISON

In [50]:
# EVALUATION
models = {
    "Naive Bayes": pred_nb,
    "Decision Tree": pred_dt,
    "SVM": pred_svm,
    "Random Forest": pred_rf
}

print("\n====== MODEL PERFORMANCE COMPARISON ======\n")
for name, pred in models.items():
    print(f"📌 {name}")
    print("Accuracy:", round(accuracy_score(y_test, pred), 4))
    print(classification_report(y_test, pred))
    print("-" * 50)

# OPTIONAL: CONFUSION MATRIX
print("\nConfusion Matrices:")
for name, pred in models.items():
    cm = confusion_matrix(y_test, pred)
    print(f"\n{name}:\n{cm}")


====== MODEL PERFORMANCE COMPARISON ======

📌 Naive Bayes
Accuracy: 0.6542
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        13
         1.0       0.20      0.12      0.15        24
         2.0       0.44      0.61      0.51        41
         3.0       0.40      0.27      0.32        37
         4.0       0.83      0.95      0.89       125

    accuracy                           0.65       240
   macro avg       0.37      0.39      0.37       240
weighted avg       0.59      0.65      0.61       240

--------------------------------------------------
📌 Decision Tree
Accuracy: 0.6875
              precision    recall  f1-score   support

         0.0       0.30      0.23      0.26        13
         1.0       0.35      0.33      0.34        24
         2.0       0.63      0.63      0.63        41
         3.0       0.44      0.38      0.41        37
         4.0       0.85      0.91      0.88       125

    accuracy             

C:\Users\Rakasya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Rakasya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Rakasya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics

In [51]:
from sklearn.feature_selection import mutual_info_classif

info = mutual_info_classif(X, y, random_state=1)

info_fitur = pd.DataFrame({"Fitur": X.columns, "Informasi": info})
info_fitur = info_fitur.sort_values(by="Informasi")

print(info_fitur)

                Fitur  Informasi
1                 Age   0.000000
5     StudyTimeWeekly   0.000000
12       Volunteering   0.000000
10             Sports   0.000000
2              Gender   0.006539
4   ParentalEducation   0.007337
3           Ethnicity   0.015724
8     ParentalSupport   0.018741
11              Music   0.019450
9     Extracurricular   0.024216
0           StudentID   0.031238
7            Tutoring   0.042196
6            Absences   0.481576


## Feature Selection

In [52]:
# FEATURE SELECTION
cols_to_drop = ["Age", "StudyTimeWeekly", "Volunteering", "Sports", "GradeClass"]

df_new = df.drop(columns=cols_to_drop)

X = df_new.drop("GPA", axis=1)  # hapus features

# DATA SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [53]:
X.head()

,StudentID,Gender,Ethnicity,ParentalEducation,Absences,Tutoring,ParentalSupport,Extracurricular,Music
0,1001,1,0,2,7,1,2,0,1
1,1002,0,0,1,0,0,1,0,0
2,1003,0,2,3,26,0,2,0,0
3,1004,1,0,3,14,0,3,1,0
4,1005,1,0,2,17,1,3,0,0


In [54]:
# MODEL TRAINING
# Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)
pred_nb = model_nb.predict(X_test_scaled)

# Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)
pred_dt = model_dt.predict(X_test)

# SVM
model_svm = SVC(kernel='rbf', C=1, gamma='scale')
model_svm.fit(X_train_scaled, y_train)
pred_svm = model_svm.predict(X_test_scaled)

# Random Forest
model_rf = RandomForestClassifier(n_estimators=150, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)

# EVALUATION
models = {
    "Naive Bayes": pred_nb,
    "Decision Tree": pred_dt,
    "SVM": pred_svm,
    "Random Forest": pred_rf
}

print("\n====== MODEL PERFORMANCE COMPARISON ======\n")
for name, pred in models.items():
    print(f"📌 {name}")
    print("Accuracy:", round(accuracy_score(y_test, pred), 4))
    print(classification_report(y_test, pred))
    print("-" * 50)

# OPTIONAL: CONFUSION MATRIX
print("\nConfusion Matrices:")
for name, pred in models.items():
    cm = confusion_matrix(y_test, pred)
    print(f"\n{name}:\n{cm}")


====== MODEL PERFORMANCE COMPARISON ======

📌 Naive Bayes
Accuracy: 0.6708
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        13
         1.0       0.28      0.29      0.29        24
         2.0       0.49      0.56      0.52        41
         3.0       0.44      0.32      0.38        37
         4.0       0.84      0.95      0.89       125

    accuracy                           0.67       240
   macro avg       0.41      0.43      0.42       240
weighted avg       0.62      0.67      0.64       240

--------------------------------------------------
📌 Decision Tree
Accuracy: 0.6417
              precision    recall  f1-score   support

         0.0       0.33      0.31      0.32        13
         1.0       0.32      0.50      0.39        24
         2.0       0.47      0.37      0.41        41
         3.0       0.40      0.32      0.36        37
         4.0       0.87      0.89      0.88       125

    accuracy             

C:\Users\Rakasya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Rakasya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Rakasya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics

In [58]:
# print("Naive Bayes")
# print("Accuracy:", round(accuracy_score(y_test, pred_nb), 4))
# print(classification_report(y_test, pred_nb))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_nb)
# print("Naive Bayes")
# print(cm)

# print("Random Forest")
# print("Accuracy:", round(accuracy_score(y_test, pred_rf), 4))
# print(classification_report(y_test, pred_rf))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_rf)
# print("Random Forest")
# print(cm)

# print("SVM")
# print("Accuracy:", round(accuracy_score(y_test, pred_svm), 4))
# print(classification_report(y_test, pred_svm))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_svm)
# print("SVM")
# print(cm)

# print("Decision Tree")
# print("Accuracy:", round(accuracy_score(y_test, pred_dt), 4))
# print(classification_report(y_test, pred_dt))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_dt)
# print("Decision Tree")
# print(cm)